In [42]:
%load_ext autoreload
%autoreload 2


import os
os.chdir('C:/Users/samet.marasli/projects/fraud_detection')
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from sklearn.pipeline import Pipeline, FeatureUnion
from xgboost.sklearn import XGBClassifier
import src.transformers as trans


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from src.pipeline import model_pipeline
import pipeline_config

In [44]:
pipeline_config.TARGET

'isFraud'

In [106]:
df_train = pd.read_csv(pipeline_config.TRAIN_DATA, nrows=1e5)
y_train = df_train.pop(pipeline_config.TARGET)
X_train = df_train


df_test = pd.read_csv(pipeline_config.TEST_DATA,nrows=5e5)
y_test = df_test.pop(pipeline_config.TARGET)
X_test = df_test


In [134]:
model_pipeline = Pipeline([
    ('drop_columns', trans.KeepFeatures(pipeline_config.FEATURES)),
    ('label_encode', trans.CustomLabelEncoder(pipeline_config.LABEL_ENCODE_FEATURES)),
    ('amount_vs_old_new_balance_orig', trans.AmountVsOldAndNewBalanceOrig()),
    ('amount_vs_old_new_balance_dest', trans.AmountVsOldAndNewBalanceDest()),
])


pipe = Pipeline(steps=[("transformation", model_pipeline), ("xgb", XGBClassifier(n_jobs = -1))])



from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    "xgb__max_depth": [1, 3],
    "xgb__scale_pos_weight": [1, 9],
}

search = GridSearchCV(pipe, param_grid, n_jobs=-1, scoring='f1', cv=3, verbose=4)
search.fit(X_train, y_train,)

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best parameter (CV score=0.785):
{'xgb__max_depth': 1, 'xgb__scale_pos_weight': 1}


In [130]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
THRESHOLD = .5
probabilities = search.best_estimator_.predict_proba(X_test)

print('F1 Score = {}'.format(f1_score(y_test, probabilities[:, 1]>THRESHOLD))) 
# print('Precision Score = {}'.format(precision_score(y_test, probabilities[:, 1]>THRESHOLD))) 
# print('Recall Score = {}'.format(recall_score(y_test, probabilities[:, 1]>THRESHOLD))) 
# print(confusion_matrix(y_test,probabilities[:, 1]>THRESHOLD)) 


F1 Score = 0.9925017041581459
Precision Score = 0.9986282578875172
Recall Score = 0.986449864498645
[[499261      1]
 [    10    728]]


----

In [188]:
import json

def json_to_dataframe(path):
	data = []
	with open(path, "r") as f:
		for line in f:
			data.append(pd.DataFrame(json.loads(line)))
	return pd.concat(data)

json_to_dataframe('./data/stream_data.txt')

,id,step,type,amount,nameOrig,oldBalanceOrig,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,1051947,201,CASH_OUT,61458.69,C681248410,22199.0,0.0,C1998294211,270625.35,332084.04,0,0
1,1048859,201,CASH_OUT,123971.71,C27885663,20954.0,0.0,C1707815182,24605.72,148577.43,0,0
2,1051607,201,TRANSFER,31230.57,C1332364972,12962.0,0.0,C641304340,0.00,31230.57,0,0
3,1047631,201,CASH_OUT,347615.00,C134684171,56672.0,0.0,C1578435326,0.00,347615.00,0,0
4,1051007,201,CASH_OUT,245849.32,C1125221187,12696.0,0.0,C327516020,91835.00,326453.49,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,1216854,213,CASH_OUT,152620.19,C725064185,50196.0,0.0,C881288996,0.00,152620.19,0,0
1,1219737,213,CASH_OUT,235098.04,C480865114,62305.0,0.0,C633340661,144661.37,379759.41,0,0
2,1217926,213,CASH_OUT,219351.46,C377015918,0.0,0.0,C923508179,9991071.34,10210422.80,0,0
3,1219630,213,CASH_OUT,148669.62,C1754299498,0.0,0.0,C1864905773,1461850.80,1610520.42,0,0


In [186]:
import json


def load_new_training_data(path):
	data = []
	with open(path, "r") as f:
		for line in f:
			data.append(pd.DataFrame(json.loads(line)))
	return pd.concat(data)

In [187]:
data2 = []
with open('predictions_deneme.txt', "r") as f:
    for line in f:
        data2.extend(json.loads(line))

FileNotFoundError: [Errno 2] No such file or directory: 'predictions_deneme.txt'

In [413]:
def store_predictions(message, model_name):
    message_file = f'predictions_{model_name}.txt'
    with open(message_file, "a") as f:
        f.write("%s\n" % (json.dumps(message)))

append_message_db(x, 'deneme')

In [142]:

df_new = load_new_training_data('./messages.txt')

In [360]:
model = pipe[-1]


In [356]:
# model_pipeline = Pipeline([
#     ('keep_features', transformers.KeepFeatures(config["FEATURES"])),
#     ('label_encode', transformers.CustomLabelEncoder(config["LABEL_ENCODE_FEATURES"])),
#     # ('amount_vs_old_new_balance_dest', transformers.AmountVsOldAndNewBalanceDest()),
#     ('amount_vs_old_new_balance_orig', transformers.AmountVsOldAndNewBalanceOrig()),
#     ('xgbclassifier', XGBClassifier(max_depth = 2, scale_pos_weight = 99, n_jobs = -1)) ,
#     ])

# df_train = pd.read_csv(config['TRAIN_DATA'])


# precisions = []
# recalls = []
# recalls2 = []


# K = 2
# train = df_train.query('step<@K')
# y_train = train.pop(config["TARGET"])
# X_train = train

# initial_model = XGBClassifier(max_depth = 2, scale_pos_weight = 99, n_jobs = -1)
# initial_model.fit(model_pipeline[:-1].transform(X_train), y_train)

# simple_model = XGBClassifier(max_depth = 2, scale_pos_weight = 99, n_jobs = -1)
# simple_model.fit(model_pipeline[:-1].transform(X_train), y_train)
# for K in range(3,300):
#     print(K)

#     train = df_train.query('step==@K')
#     test = df_train.query('step==@K+1')
    
#     y_train = train.pop(config["TARGET"])
#     X_train = train

#     y_test = test.pop(config["TARGET"])
#     X_test = test

#     # print(X_train.shape, X_test.shape)

#     try:
#         # retrain_model = XGBClassifier(max_depth = 2, scale_pos_weight = 99, n_jobs = -1)
#         initial_model.fit(model_pipeline[:-1].transform(X_train), y_train, xgb_model= initial_model)
        
#         probabilities = initial_model.predict_proba(model_pipeline[:-1].transform(X_test))
#         precisions.append(precision_score(y_test, probabilities[:, 1]>0.5))
#         recalls.append(recall_score(y_test, probabilities[:, 1]>0.5))
        

#         probabilities2 = simple_model.predict_proba(model_pipeline[:-1].transform(X_test))
#         recalls2.append(recall_score(y_test, probabilities2[:, 1]>0.5))

#     except:
#         pass

#     print('----')


In [357]:
# import matplotlib.pyplot as plt
# # plt.plot(precisions)
# plt.figure(figsize=(30,10))
# plt.plot(recalls,color='r', alpha=.4)
# plt.plot(recalls2,color='black', alpha=.4)

In [451]:
from time import sleep

In [454]:
while True:
    sleep(2)
    datav1 = []
    with open('predictions_xgb_v1.txt', "r") as f:
        for line in f:
            datav1.append(json.loads(line))

    datav2 = []
    with open('predictions_xgb_v2.txt', "r") as f:
        for line in f:
            datav2.append(json.loads(line))
    datav1 = pd.DataFrame(datav1)
    datav2 = pd.DataFrame(datav2)
    print(sum(datav1['request_id'].isin(datav2['request_id'])))


260
261


KeyboardInterrupt: 